# [Oregon Statewide ET Project](https://www.dri.edu/project/owrd-et/)


## **Google Earth Engine (GEE) Zonal Stats Exports**
> Export to Google Cloud Storage or Google Drive<br>
> Export each year's summary separately as composite dataframes (similar to an ArcGIS attribute table for a shapefile)<br>
> Monthly, annual (irrigation status), and static (HUC attributes) summaries for ~250,000 features
 
## Field-level export types: 
1. **monthly** (OpenET Ensemble and gridMET)
2. **climatologies** (OpenET Ensemble and gridMET)
   * Start/End year options for the gap-filling windows:
    > 1984-1991<br>
    > 1992-1997<br>
    > 1998-2003<br>
    > 2004-2009<br>
    > 2010-2015<br>
    > 2016-2021
4. **annual** (Irrigation Status; IrrMapper and EToF)
5. **static** (HUC8 and HUC12 attributes, centroid-based) summaries
<br>


--------

In [23]:
# ----------------------------PARAMETERS---------------------------------------------------------

# you must use a registered google cloud project to enable GEE access
gcloud_project_id = 'ee-bminor'

# export data to cloud_storage or google_drive
export_location = 'google_drive'

### ONLY USED IF EXPORTING DATA TOa CLOUD STORAGE
# path to cloud storage bucket if exporting data to cloud storage
gcloud_path = "openet/intercomparison/output_main/Oregon_Statewide_2023/field_summaries/historical"


# flag for testing an export for a single field (low compute and storage)
test_flag = True

# define start/end years for exports
start_year = 1985
end_year = 1991


# -----------------------------------------------------------------------------------------------

#### Import and initialize the GEE API

In [2]:
import ee
# ee.Authenticate()
ee.Initialize(project=gcloud_project_id)

## 1. **monthly** OpenET and gridMET
### First, set the export variable:
> 'et': OpenET Ensemble actual ET<br>
> 'et_reference': Bias-corrected gridMET reference ET (ETo)<br>
> 'et_fraction': Fraction of reference ET (EToF)<br>
> 'ppt': gridMET total Precipitation<br>

In [21]:
# -------------------------------------------------------------------------------------

# specify which variable to export here
variable = 'et'

# -------------------------------------------------------------------------------------

#### Compute and export field-level zonal stats (spatial mean) using weighted reducer

In [ ]:
# function to compute the field area (acres approximate, sq. m divided by 4047) and sets property
def calcArea(ftr):
    return ftr.set({
        'ACRES_FTR_GEOM_EE': ftr.geometry().area().divide(4047),
    })

# function to add date properties to images
def addDates(img):
    img_date = ee.Date(img.get('system:time_start'))
    return img.set('date',img_date.format('yyyy-MM-dd'))

# simple join function for images
def joinFunc(img):
    return ee.Image.cat(img.get('primary'), img.get('secondary'))

# function to calculate fraction reference ET
def calcETF(img):
    img = img.addBands(img.select('et').divide(img.select('et_reference'))).rename(['et', 'et_reference', 'et_fraction'])
    return img.select(['et_fraction'])
    
# function to remove field-boundary geometries from zonal stats output
def removGeom(ftr):
    return ftr.setGeometry(None)


# start/end dates for the statewide et project
study_start = '1984-11-01'
study_end = '2022-11-01' # exclusive

# dictionary containg variables (keys) and output variable names/dataset source assetIDs (values) that are used
dataset_dict = {
    'et': ['ETa', 'projects/openet/assets/ensemble/conus/gridmet/monthly/provisional', 'OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0'],
    'et_reference': ['ET_Reference', 'projects/openet/assets/reference_et/conus/gridmet/monthly/v1'],
    'et_fraction': ['ET_Fraction', 'projects/openet/assets/ensemble/conus/gridmet/monthly/provisional', 'OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0', 'projects/openet/assets/reference_et/conus/gridmet/monthly/v1'],
    'ppt': ['PPT', 'IDAHO_EPSCOR/GRIDMET'],
}


# list of years to process based on start/end year parameters
potential_year_list = list(range(1985, 2023))
if start_year > end_year:
    print('end_year cannot be less than start_year, please check the parameters')
if start_year not in potential_year_list:
    print('start_year is not within the 1985-2022 study period')
if end_year not in potential_year_list:
    print('end_year is not within the 1985-2022 study period')
year_list = list(range(start_year, end_year+1))


# field boundary assetID on GEE
if test_flag:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
            .filter(ee.Filter.eq('OPENET_ID', 'ORx_155121'))
    )
else:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
    )
    
# map the area function over the field boundary feature collection
field_bound = field_bound_pre.map(calcArea)

# prep image collections
if variable == 'et':

    # Nov 1984 - Sept 1999 ET collection
    monthly_coll_1 = (
        ee.ImageCollection(dataset_dict[variable][1])
            .select(['et_ensemble_mad'], [variable])
            .filter(ee.Filter.date(study_start, '1999-10-01'))
    )

    # Oct 1999 - Sept 2022 ET collection
    monthly_coll_2 = (
        ee.ImageCollection(dataset_dict[variable][2])
            .select(['et_ensemble_mad'], [variable])
            .filter(ee.Filter.date('1999-10-01', study_end))
    )

    # merge image collections
    final_coll = monthly_coll_1.merge(monthly_coll_2)

elif variable == 'et_reference':

    # gridMET ETo collections
    final_coll = (
        ee.ImageCollection(dataset_dict[variable][1])
            .select(['eto'], [variable])
            .filter(ee.Filter.date(study_start, study_end))
    )

elif variable == 'et_fraction':

    # Nov 1984 - Sept 1999 ET collection
    monthly_coll_1 = (
        ee.ImageCollection(dataset_dict[variable][1])
            .select(['et_ensemble_mad'], ['et'])
            .filter(ee.Filter.date(study_start, '1999-10-01'))
            .map(addDates)
    )

    # Oct 1999 - Sept 2022 ET collection
    monthly_coll_2 = (
        ee.ImageCollection(dataset_dict[variable][2])
            .select(['et_ensemble_mad'], ['et'])
            .filter(ee.Filter.date('1999-10-01', study_end))
            .map(addDates)
    )

    # merge image collections
    monthly_coll = monthly_coll_1.merge(monthly_coll_2)

    # gridMET ETo collection
    gridmet_coll = (
        ee.ImageCollection(dataset_dict[variable][3])
            .select(['eto'], ['et_reference'])
            .filter(ee.Filter.date('1984-11-01', '2022-11-01'))
            .map(addDates)
    )

    # GEE filter using date properties    
    filter_c = ee.Filter.equals(leftField='date', rightField='date')

    # inner join keeps matches only
    innJoin = ee.Join.inner()

    # apply the inner join with the filter
    innerApp = ee.ImageCollection(innJoin.apply(monthly_coll, gridmet_coll, filter_c))

    # Combine the images into a single image which contains all bands from all of the images
    # also calculate EToF
    final_coll = (
        innerApp
            .map(joinFunc)
            .map(calcETF)
    )

elif variable == 'ppt':
    
    # start and end dates of the OpenET data
    Date_Start = ee.Date(study_start)
    Date_End = ee.Date(study_end)
    
    # Create list of dates for time series
    n_months = Date_End.difference(Date_Start, 'month').round()
    dates = ee.List.sequence(0, n_months, 1)
    def makeDates(n):
        return Date_Start.advance(n, 'month')
    dates = dates.map(makeDates)
    
    # daily gridMET precip collection
    en_dt_exc = Date_End.advance(1, 'month').format('yyyy-MM-dd')
    
    daily_coll = (
        ee.ImageCollection(dataset_dict[variable][1])
            .select(['pr'], [variable])
            .filter(ee.Filter.date(study_start, study_end))
    )
    
    # define a function to convert daily precip to monthly precip
    def monthColl(dat):
    
        # get earth engine date object and format as strings
        dat_obj = ee.Date(dat)
        dat_mo = dat_obj.get('month')
        dat_yr = dat_obj.get('year')
        dat_str = dat_obj.format('yyyy-MM-dd')
    
        # filter the image collection to the date
        imgs = (
            daily_coll
                .filter(ee.Filter.calendarRange(dat_yr, dat_yr, 'year'))
                .filter(ee.Filter.calendarRange(dat_mo, dat_mo, 'month'))
        )
    
        # calculate the monthly total
        img = imgs.sum()
    
        return img.set({
            'date': dat_str,
            'system:time_start': ee.Date(dat_str).millis()
        })
        
    # map the function over the dates to create the monthly image collection
    final_coll = ee.ImageCollection(dates.map(monthColl))


print(f'PROCESSING {year_list[0]} to {year_list[-1]}')
# loop through each year that is being run
for year in year_list:

    # filter collection to Nov-Oct months for the current year
    final_coll_sub = (
        final_coll
            .filter(ee.Filter.date(f'{year-1}-11-01', f'{year}-11-01'))
    )

    # monthly images
    if variable == 'et_fraction':
        var_nov = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(11, 11, 'month')).mean().rename([f'{variable}_11']))
        var_dec = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(12, 12, 'month')).mean().rename([f'{variable}_12']))
        var_jan = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(1, 1, 'month')).mean().rename([f'{variable}_01']))
        var_feb = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(2, 2, 'month')).mean().rename([f'{variable}_02']))
        var_mar = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(3, 3, 'month')).mean().rename([f'{variable}_03']))
        var_apr = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(4, 4, 'month')).mean().rename([f'{variable}_04']))
        var_may = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(5, 5, 'month')).mean().rename([f'{variable}_05']))
        var_jun = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(6, 6, 'month')).mean().rename([f'{variable}_06']))
        var_jul = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(7, 7, 'month')).mean().rename([f'{variable}_07']))
        var_aug = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(8, 8, 'month')).mean().rename([f'{variable}_08']))
        var_sep = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(9, 9, 'month')).mean().rename([f'{variable}_09']))
        var_oct = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(10, 10, 'month')).mean().rename([f'{variable}_10']))
    else:
        var_nov = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(11, 11, 'month')).sum().rename([f'{variable}_11']))
        var_dec = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(12, 12, 'month')).sum().rename([f'{variable}_12']))
        var_jan = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(1, 1, 'month')).sum().rename([f'{variable}_01']))
        var_feb = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(2, 2, 'month')).sum().rename([f'{variable}_02']))
        var_mar = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(3, 3, 'month')).sum().rename([f'{variable}_03']))
        var_apr = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(4, 4, 'month')).sum().rename([f'{variable}_04']))
        var_may = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(5, 5, 'month')).sum().rename([f'{variable}_05']))
        var_jun = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(6, 6, 'month')).sum().rename([f'{variable}_06']))
        var_jul = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(7, 7, 'month')).sum().rename([f'{variable}_07']))
        var_aug = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(8, 8, 'month')).sum().rename([f'{variable}_08']))
        var_sep = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(9, 9, 'month')).sum().rename([f'{variable}_09']))
        var_oct = ee.Image(final_coll_sub.filter(ee.Filter.calendarRange(10, 10, 'month')).sum().rename([f'{variable}_10']))

    # add all bands
    all_bnds = (
        var_nov.addBands(var_dec).addBands(var_jan).addBands(var_feb).addBands(var_mar)
            .addBands(var_apr).addBands(var_may).addBands(var_jun).addBands(var_jul)
            .addBands(var_aug).addBands(var_sep).addBands(var_oct)
    )

    # zonal stats computation (reduceRegions, spatial mean, composite dataframe output)
    stats_out = (
        all_bnds
            .reduceRegions(
                collection=field_bound,
                reducer=ee.Reducer.mean(),
                scale=30,
                tileScale=16
            )
            .map(removGeom)
    )

    # format the output
    def orgFeatColl(ftr):
        return ftr.set({
            'ACRES_FTR_GEOM': ftr.get('ACRES_FTR_GEOM_EE'),
            f'{dataset_dict[variable][0]}_11_{str(year-1)[2:]}': ftr.get(f'{variable}_11'),
            f'{dataset_dict[variable][0]}_12_{str(year-1)[2:]}': ftr.get(f'{variable}_12'),
            f'{dataset_dict[variable][0]}_01_{str(year)[2:]}': ftr.get(f'{variable}_01'),
            f'{dataset_dict[variable][0]}_02_{str(year)[2:]}': ftr.get(f'{variable}_02'),
            f'{dataset_dict[variable][0]}_03_{str(year)[2:]}': ftr.get(f'{variable}_03'),
            f'{dataset_dict[variable][0]}_04_{str(year)[2:]}': ftr.get(f'{variable}_04'),
            f'{dataset_dict[variable][0]}_05_{str(year)[2:]}': ftr.get(f'{variable}_05'),
            f'{dataset_dict[variable][0]}_06_{str(year)[2:]}': ftr.get(f'{variable}_06'),
            f'{dataset_dict[variable][0]}_07_{str(year)[2:]}': ftr.get(f'{variable}_07'),
            f'{dataset_dict[variable][0]}_08_{str(year)[2:]}': ftr.get(f'{variable}_08'),
            f'{dataset_dict[variable][0]}_09_{str(year)[2:]}': ftr.get(f'{variable}_09'),
            f'{dataset_dict[variable][0]}_10_{str(year)[2:]}': ftr.get(f'{variable}_10'),

        })

    # map the function to format 
    stats_out_form  = ee.FeatureCollection(stats_out.map(orgFeatColl))

    # list of properties to export
    if variable == 'et':
        selector_list = ['OPENET_ID', 'ACRES_FTR_GEOM',
                          f'{dataset_dict[variable][0]}_11_{str(year-1)[2:]}', f'{dataset_dict[variable][0]}_12_{str(year-1)[2:]}', f'{dataset_dict[variable][0]}_01_{str(year)[2:]}',
                          f'{dataset_dict[variable][0]}_02_{str(year)[2:]}', f'{dataset_dict[variable][0]}_03_{str(year)[2:]}', f'{dataset_dict[variable][0]}_04_{str(year)[2:]}',
                          f'{dataset_dict[variable][0]}_05_{str(year)[2:]}', f'{dataset_dict[variable][0]}_06_{str(year)[2:]}', f'{dataset_dict[variable][0]}_07_{str(year)[2:]}',
                          f'{dataset_dict[variable][0]}_08_{str(year)[2:]}', f'{dataset_dict[variable][0]}_09_{str(year)[2:]}', f'{dataset_dict[variable][0]}_10_{str(year)[2:]}',
                        ]
    else:
        selector_list = ['OPENET_ID',
                          f'{dataset_dict[variable][0]}_11_{str(year-1)[2:]}', f'{dataset_dict[variable][0]}_12_{str(year-1)[2:]}', f'{dataset_dict[variable][0]}_01_{str(year)[2:]}',
                          f'{dataset_dict[variable][0]}_02_{str(year)[2:]}', f'{dataset_dict[variable][0]}_03_{str(year)[2:]}', f'{dataset_dict[variable][0]}_04_{str(year)[2:]}',
                          f'{dataset_dict[variable][0]}_05_{str(year)[2:]}', f'{dataset_dict[variable][0]}_06_{str(year)[2:]}', f'{dataset_dict[variable][0]}_07_{str(year)[2:]}',
                          f'{dataset_dict[variable][0]}_08_{str(year)[2:]}', f'{dataset_dict[variable][0]}_09_{str(year)[2:]}', f'{dataset_dict[variable][0]}_10_{str(year)[2:]}',
                        ]
    
    # Export tasks
    if export_location == 'google_drive':

        # Export a CSV file to Google Drive.
        out_table_task = ee.batch.Export.table.toDrive(**{
            'collection': stats_out_form,
            'description': f'OR_Field_Bound_Summary_Export_{variable}_{year}',
            'fileNamePrefix': f'or_field_summaries_water_year_shift_1mo_{year}_{variable}',
            'fileFormat': 'CSV',
            'selectors': selector_list,
        })

    elif export_location == 'cloud_storage':
        
        # Export a CSV file to Cloud Storage.
        out_table_task = ee.batch.Export.table.toCloudStorage(**{
            'collection': stats_out_form,
            'description': f'OR_Field_Bound_Summary_Export_{variable}_{year}',
            'bucket': gcloud_path.split('/')[0],
            'fileNamePrefix': f'{gcloud_path}/or_field_summaries_water_year_shift_1mo_{year}_{variable}',
            'fileFormat': 'CSV',
            'selectors': selector_list,
         })

    else:
        print('wrong export location setting, please check the parameter')
        continue

    # start the task
    out_table_task.start()

    # print the status of the task
    # print(out_table_task.status()["id"])
    
    print(f'task started for {variable} {year}')

## 2. **climatologies** (EToF)

* Start/End Year Options:
> 1984-1991 (note: 1984 included for climo b/c of high cloud cover and 1-Landsat years in this window causing missing ET months)<br>
> 1992-1997<br>
> 1998-2003<br>
> 2004-2009<br>
> 2010-2015<br>
> 2016-2021

In [8]:
# -------------------------------------------------------------------------------------

# specify which variable to export here
variable = 'et_fraction_climo'

# -------------------------------------------------------------------------------------

In [ ]:
# function to compute the field area (acres approximate, sq. m divided by 4047) and sets property
def calcArea(ftr):
    return ftr.set({
        'ACRES_FTR_GEOM_EE': ftr.geometry().area().divide(4047),
    })

# function to add date properties to images
def addDates(img):
    img_date = ee.Date(img.get('system:time_start'))
    return img.set('date',img_date.format('yyyy-MM-dd'))

# simple join function for images
def joinFunc(img):
    return ee.Image.cat(img.get('primary'), img.get('secondary'))

# function to calculate fraction reference ET
def calcETF(img):
    img = img.addBands(img.select('et').divide(img.select('et_reference'))).rename(['et', 'et_reference', 'et_fraction'])
    return img.select(['et_fraction'])
    
# function to remove field-boundary geometries from zonal stats output
def removGeom(ftr):
    return ftr.setGeometry(None)

# dictionary containg variables (keys) and output variable names/dataset source assetIDs (values) that are used
dataset_dict = {
    'et_fraction_climo': ['ET_Fraction', 'projects/openet/assets/ensemble/conus/gridmet/monthly/provisional', 'OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0', 'projects/openet/assets/reference_et/conus/gridmet/monthly/v1'],
}


# list of years to process based on start/end year parameters
potential_year_list = list(range(1984, 2022))
if start_year > end_year:
    print('end_year cannot be less than start_year, please check the parameters')
if start_year not in potential_year_list:
    print('start_year is not within the 1984-2021 climatology years')
if end_year not in potential_year_list:
    print('end_year is not within the 1984-2021 climatology years')
year_list = list(range(start_year, end_year+1))


# field boundary assetID on GEE
if test_flag:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
            .filter(ee.Filter.eq('OPENET_ID', 'ORx_155121'))
    )
else:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
    )
    
# map the area function over the field boundary feature collection
field_bound = field_bound_pre.map(calcArea)



if variable == 'et_fraction_climo':

    # Nov 1984 - Sept 1999 ET collection
    monthly_coll_1 = (
        ee.ImageCollection(dataset_dict[variable][1])
            .select(['et_ensemble_mad'], ['et'])
            .filter(ee.Filter.date('1984-03-01', '1999-10-01'))
            .map(addDates)
    )

    # Oct 1999 - Sept 2022 ET collection
    monthly_coll_2 = (
        ee.ImageCollection(dataset_dict[variable][2])
            .select(['et_ensemble_mad'], ['et'])
            .filter(ee.Filter.date('1999-10-01', '2021-11-01'))
            .map(addDates)
    )

    # merge image collections
    monthly_coll = monthly_coll_1.merge(monthly_coll_2)

    # gridMET ETo collection
    gridmet_coll = (
        ee.ImageCollection(dataset_dict[variable][3])
            .select(['eto'], ['et_reference'])
            .filter(ee.Filter.date('1984-03-01', '2021-11-01'))
            .map(addDates)
    )

    # GEE filter using date properties    
    filter_c = ee.Filter.equals(leftField='date', rightField='date')

    # inner join keeps matches only
    innJoin = ee.Join.inner()

    # apply the inner join with the filter
    innerApp = ee.ImageCollection(innJoin.apply(monthly_coll, gridmet_coll, filter_c))

    # Combine the images into a single image which contains all bands from all of the images
    # also calculate EToF
    final_coll = (
        innerApp
            .map(joinFunc)
            .map(calcETF)
    )

    # filter to the climatology window that was defined by the start/end years
    final_coll_window = (
        final_coll
            .filter(ee.Filter.date(f'{start_year-1}-11-01', f'{end_year}-11-01'))
    )

else:
    print('wrong variable selected for this script, can only select et_fraction_climo')


var_nov = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(11, 11, 'month')).mean().rename([f'{variable}_11']))
var_dec = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(12, 12, 'month')).mean().rename([f'{variable}_12']))
var_jan = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(1, 1, 'month')).mean().rename([f'{variable}_01']))
var_feb = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(2, 2, 'month')).mean().rename([f'{variable}_02']))
var_mar = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(3, 3, 'month')).mean().rename([f'{variable}_03']))
var_apr = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(4, 4, 'month')).mean().rename([f'{variable}_04']))
var_may = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(5, 5, 'month')).mean().rename([f'{variable}_05']))
var_jun = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(6, 6, 'month')).mean().rename([f'{variable}_06']))
var_jul = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(7, 7, 'month')).mean().rename([f'{variable}_07']))
var_aug = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(8, 8, 'month')).mean().rename([f'{variable}_08']))
var_sep = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(9, 9, 'month')).mean().rename([f'{variable}_09']))
var_oct = ee.Image(final_coll_window.filter(ee.Filter.calendarRange(10, 10, 'month')).mean().rename([f'{variable}_10']))

# add all bands
all_bnds = (
    var_nov.addBands(var_dec).addBands(var_jan).addBands(var_feb).addBands(var_mar)
        .addBands(var_apr).addBands(var_may).addBands(var_jun).addBands(var_jul)
        .addBands(var_aug).addBands(var_sep).addBands(var_oct)
)

# zonal stats computation (reduceRegions, spatial mean, composite dataframe output)
stats_out = (
    all_bnds
        .reduceRegions(
            collection=field_bound,
            reducer=ee.Reducer.mean(),
            scale=30,
            tileScale=16
        )
        .map(removGeom)
)

# format the output
def orgFeatColl(ftr):
    return ftr.set({
        'ACRES_FTR_GEOM': ftr.get('ACRES_FTR_GEOM_EE'),
        f'{dataset_dict[variable][0]}_11': ftr.get(f'{variable}_11'),
        f'{dataset_dict[variable][0]}_12': ftr.get(f'{variable}_12'),
        f'{dataset_dict[variable][0]}_01': ftr.get(f'{variable}_01'),
        f'{dataset_dict[variable][0]}_02': ftr.get(f'{variable}_02'),
        f'{dataset_dict[variable][0]}_03': ftr.get(f'{variable}_03'),
        f'{dataset_dict[variable][0]}_04': ftr.get(f'{variable}_04'),
        f'{dataset_dict[variable][0]}_05': ftr.get(f'{variable}_05'),
        f'{dataset_dict[variable][0]}_06': ftr.get(f'{variable}_06'),
        f'{dataset_dict[variable][0]}_07': ftr.get(f'{variable}_07'),
        f'{dataset_dict[variable][0]}_08': ftr.get(f'{variable}_08'),
        f'{dataset_dict[variable][0]}_09': ftr.get(f'{variable}_09'),
        f'{dataset_dict[variable][0]}_10': ftr.get(f'{variable}_10'),

    })

# map the function to format 
stats_out_form  = ee.FeatureCollection(stats_out.map(orgFeatColl))

# list of properties to export
selector_list = ['OPENET_ID',
                  f'{dataset_dict[variable][0]}_11', f'{dataset_dict[variable][0]}_12', f'{dataset_dict[variable][0]}_01',
                  f'{dataset_dict[variable][0]}_02', f'{dataset_dict[variable][0]}_03', f'{dataset_dict[variable][0]}_04',
                  f'{dataset_dict[variable][0]}_05', f'{dataset_dict[variable][0]}_06', f'{dataset_dict[variable][0]}_07',
                  f'{dataset_dict[variable][0]}_08', f'{dataset_dict[variable][0]}_09', f'{dataset_dict[variable][0]}_10',
                ]

# Export tasks
if export_location == 'google_drive':

    # Export a CSV file to Google Drive.
    out_table_task = ee.batch.Export.table.toDrive(**{
        'collection': stats_out_form,
        'description': f'OR_Field_Bound_Summary_Export_{variable}_{start_year}_{end_year}',
        'fileNamePrefix': f'or_field_summaries_water_year_shift_1mo_{start_year}_{end_year}_{variable}',
        'fileFormat': 'CSV',
        'selectors': selector_list,
    })

elif export_location == 'cloud_storage':
    
    # Export a CSV file to Cloud Storage.
    out_table_task = ee.batch.Export.table.toCloudStorage(**{
        'collection': stats_out_form,
        'description': f'OR_Field_Bound_Summary_Export_{variable}_{start_year}_{end_year}',
        'bucket': gcloud_path.split('/')[0],
        'fileNamePrefix': f'{gcloud_path}/or_field_summaries_water_year_shift_1mo_{start_year}_{end_year}_{variable}',
        'fileFormat': 'CSV',
        'selectors': selector_list,
     })

else:
    print('wrong export location setting, please check the parameter')

# start the task
out_table_task.start()

# print the status of the task
# print(out_table_task.status()["id"])

print(f'task started for {variable} {start_year} - {end_year}')

## 3. **annual** Irrigation Status (IrrMapper and EToF)
### First, set the export variable:
> 'irrmapper_irrigated': IrrMapper Irrigated Class<br>
> 'irrmapper_wetland': IrrMapper Wetland Class<br>
> 'etof_irr_status': EToF irrigation status classification (1 - Not Irrigated, 2 - Irrigated, 3 - Shorted)

In [28]:
# -------------------------------------------------------------------------------------

# specify which variable to export here
variable = 'irrmapper_irrigated'

# -------------------------------------------------------------------------------------

In [ ]:
# function to compute the field area (acres approximate, sq. m divided by 4047) and sets property
def calcArea(ftr):
    return ftr.set({
        'ACRES_FTR_GEOM_EE': ftr.geometry().area().divide(4047),
    })

# function to add date properties to images
def addDates(img):
    img_date = ee.Date(img.get('system:time_start'))
    return img.set('date',img_date.format('yyyy-MM-dd'))

# simple join function for images
def joinFunc(img):
    return ee.Image.cat(img.get('primary'), img.get('secondary'))
    
# function to remove field-boundary geometries from zonal stats output
def removGeom(ftr):
    return ftr.setGeometry(None)


# start/end dates for the statewide et project
study_start = '1984-11-01'
study_end = '2022-11-01' # exclusive

# dictionary containg variables (keys) and pixel class values/output variable names/dataset source assetIDs (values) that are used
dataset_dict = {
    'irrmapper_irrigated': [0, 'IRRIGATED', 'projects/ee-dgketchum/assets/IrrMapper/IrrMapperComp'],
    'irrmapper_wetland': [3, 'WETLAND', 'projects/ee-dgketchum/assets/IrrMapper/IrrMapperComp'],
    'etof_irr_status': ['ETOF_IRR_STATUS_MODE', 'projects/openet/assets/ensemble/conus/gridmet/monthly/provisional', 'OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0', 'projects/openet/assets/reference_et/conus/gridmet/monthly/v1'],
}


# full list of years within the study period
potential_year_list = list(range(1985, 2023))

if start_year > end_year:
    print('end_year cannot be less than start_year, please check the parameters')
if start_year not in potential_year_list:
    print('start_year is not within the 1985-2022 study period')
if end_year not in potential_year_list:
    print('end_year is not within the 1985-2022 study period')

# list of years to process based on start/end year parameters
year_list = list(range(start_year, end_year+1))


# field boundary assetID on GEE
if test_flag:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
            .filter(ee.Filter.eq('OPENET_ID', 'ORx_155121'))
    )
else:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
    )
    
# map the area function over the field boundary feature collection
field_bound = field_bound_pre.map(calcArea)

print(f'PROCESSING {year_list[0]} to {year_list[-1]}')
for year in year_list:
    
    # prep image collections
    if (variable == 'irrmapper_irrigated' or variable == 'irrmapper_wetland'):

        
        # IrrMapper Irrigated class image for the current year
        irr = (
            ee.ImageCollection(dataset_dict[variable][2])
                .filter(ee.Filter.calendarRange(year, year, 'year'))
                .select('classification')
                .max()
                .rename([dataset_dict[variable][1]])
        )

        # pixel values of 0 is irrigated class, 3 is wetland class; first build a mask of those values
        mask = irr.eq(dataset_dict[variable][0])

        # mask the image to irrigated pixels only
        irrmapper_mask = irr.updateMask(mask).remap([dataset_dict[variable][0]], [1])

        # Nov 1984 - Sept 1999 ET collection
        monthly_coll_1 = (
            ee.ImageCollection('projects/openet/assets/ensemble/conus/gridmet/monthly/provisional')
                .select(['et_ensemble_mad'], ['et'])
                .filter(ee.Filter.date(study_start, '1999-10-01'))
        )
    
        # Oct 1999 - Sept 2022 ET collection
        monthly_coll_2 = (
            ee.ImageCollection('OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0')
                .select(['et_ensemble_mad'], ['et'])
                .filter(ee.Filter.date('1999-10-01', study_end))
        )
    
        # merge image collections
        final_coll = monthly_coll_1.merge(monthly_coll_2)

        # make an annual image from monthly
        eta_an = (
            ee.Image(
                final_coll
                    .select('et')
                    .filter(ee.Filter.calendarRange(year, year, 'year'))
                    .sum()
            )
            .rename(['eta_an'])
        )

        # make an annual image from monthly but masked to irrigated pixels only
        eta_an_masked = (
            ee.Image(
                final_coll
                    .select('et')
                    .filter(ee.Filter.calendarRange(year, year, 'year'))
                    .sum()
            )
            .updateMask(irrmapper_mask)
            .rename(['eta_an'])
        )
    
        # Image for computing area (ACRES_FTR_GEOM) of all pixels within field boundary
        area_all_img = (
            eta_an
                .divide(eta_an)
                .multiply(ee.Image.pixelArea())
                .rename(['ACRES_ALL_PIXELS'])
                .divide(4047) 
        )
    
        # Image for computing area (ACRES_FTR_GEOM) of irrigated/wetland pixels within field boundary (uses the irr_mask image as a mask)
        area_masked_img = (
            eta_an_masked
                .divide(eta_an_masked)
                .multiply(ee.Image.pixelArea())
                .rename([f'ACRES_{dataset_dict[variable][1]}_PIXELS'])
                .divide(4047)
        )
    
        # stack bands  
        all_bnds = area_all_img.addBands(area_masked_img)

        # zonal stats computation (reduceRegions, spatial sum, composite dataframe output)
        stats_out = (
            all_bnds
                .reduceRegions(
                    collection=field_bound,
                    reducer=ee.Reducer.sum(),
                    scale=30,
                    tileScale=16
                )
                .map(removGeom)
        )
    
        # format the output
        def orgFeatColl(ftr):
            return ftr.set({
                'ACRES_ALL': ftr.get('ACRES_ALL_PIXELS'),
                f'ACRES_{dataset_dict[variable][1]}': ftr.get(f'ACRES_{dataset_dict[variable][1]}_PIXELS'),
            })
    
        # map the function to organize 
        stats_out_form = ee.FeatureCollection(stats_out.map(orgFeatColl))

    
    elif variable == 'etof_irr_status':

        # Identify months to run
        start_month = "05"
        end_month = "10"
        
        # Shorted slope months
        short_start_month = "06"
        short_end_month = "09"
        
        # Define EToF threshold
        etof_threshold = 0.5
        etof_short_threshold = -0.05

        stats_out = ee.FeatureCollection([])
        
        # Nov 1984 - Sept 1999 ET collection
        monthly_coll_1 = (
            ee.ImageCollection(dataset_dict[variable][1])
                .select(['et_ensemble_mad'], ['et'])
                .filter(ee.Filter.date(study_start, '1999-10-01'))
                .map(addDates)
        )
        
        # Oct 1999 - Sept 2022 ET collection
        monthly_coll_2 = (
            ee.ImageCollection(dataset_dict[variable][2])
                .select(['et_ensemble_mad'], ['et'])
                .filter(ee.Filter.date('1999-10-01', study_end))
                .map(addDates)
        )
        
        # merge image collections
        monthly_coll = monthly_coll_1.merge(monthly_coll_2)
    
    
        # -------------------------------- Build monthly EToF collection -----------------------------
        
    
        # Set start and end dates and create months sequence to iterate over
        # Run for 2016-2021
        start_date = ee.Date(str(year) + '-' + start_month + '-01') # set analysis start date
        end_date = ee.Date(str(year) + '-' + end_month + '-01') # set analysis end date
        count_months = ee.Number(end_date.difference(start_date, 'month')).round()
        month_list = ee.List.sequence(0, count_months)
    
        # Function to create monthly ETOF image collection
        def createMonthlyEtof(i):
    
            # Calculate the offset from start_date and create eng
            ini = start_date.advance(i, 'month')
            end = ini.advance(1, 'month')
    
            # Get first image from collection
            first_img = (
                ee.Image(monthly_coll
                    .filterDate(ini, end)
                    .select(['et'])
                    .first())
            )
    
            # Filter and reduce the EE Metric Collection to median monthly composite
            et = (
                monthly_coll
                    .filterDate(ini, end)
                    .select(['et'])
                    .median()
            )
    
            # Filter and get first ETO image 
            eto = ee.ImageCollection(dataset_dict[variable][3]).filterDate(ini,end).select(['eto']).first()
    
            # Calculate ETOF for the month
            etof = ee.Image(et).divide(eto).rename('etof')
    
            return ee.Image(ee.Image(etof).copyProperties(first_img, ['system:index', 'system:time_start', 'start_date', 'end_date']))
    
        # Map EToF function over images
        month_etof_ic = ee.ImageCollection(month_list.map(createMonthlyEtof))
    
        # ------------------------------- Apply EToF threshold ----------------------------------------
    
    
        # Apply function to get binary ETOF monthly image collection of ETOF >0.5
        def createMonthlyBinaryEtof(i):
            return(i.gte(etof_threshold).set(i.toDictionary()).set('system:time_start', i.get('system:time_start')))
    
        # Map binary EToF function over images
        month_etof_ic_bi = ee.ImageCollection(month_etof_ic).map(createMonthlyBinaryEtof)
    
    
        # --------------------- Assign labels based on monthly EToF time-series -----------------------
    
        # Label categories and definitions
        # 1: Not Irrigated: EToF < 0.5 for most of all growing season months (4 or greater months).
        # 2: Irrigated: EToF > 0.5 May-October (3+ months)
        # 3: Shorted: Doesn't meet criteria for Irrigated or Not Irrigated AND EToF slope of < -0.05
        # 4: Other: Max extent lands not classified using this schema
    
        # Generate irrigated and not irrigated classes based on ruleset described above
        irrigated_i = month_etof_ic_bi.sum().gte(3)
        notirrigated_i = month_etof_ic_bi.sum().lte(2)
    
        # Generate shorted by masking irrigated and not irrigated pixels and applying sens slope reducer and threshold
        def slopePrep(img):
            img = ee.Image(img)
            month = ee.Number.parse(ee.String(img.get('start_date')).slice(5, 7))
            img_month = ee.Image(month).rename('month').toInt()
            return (img.addBands(img_month))
        
        month_etof_slope = (
            month_etof_ic
                .filterDate(str(year) + '-' + short_start_month + '-01', str(year) + '-' + short_end_month + '-30')
                .map(slopePrep)
                .select('month', 'etof')
                .reduce(ee.Reducer.sensSlope())
                .select('slope')
        )
    
        shorted_i = irrigated_i.updateMask(month_etof_slope.lt(etof_short_threshold))
    
        # Generate other by masking all classified pixels
        other_i = month_etof_ic.first().neq(-100).updateMask(irrigated_i.Not()).updateMask(notirrigated_i.Not()).updateMask(shorted_i.Not()) 
        
        class_ls = ee.ImageCollection([notirrigated_i.multiply(1).toByte().selfMask(),
                                             irrigated_i.multiply(2).toByte().selfMask(),
                                             shorted_i.multiply(3).toByte().selfMask(),
                                             other_i.multiply(4).toByte().selfMask()]).select([0], ['class'])
        
    
        # image used to compute mode of values for each field
        irr_all_c = class_ls.mosaic().rename(['irr_status_all_c'])
        
        irr_1_m = irr_all_c.eq(1)
        irr_2_m = irr_all_c.eq(2)
        irr_3_m = irr_all_c.eq(3)
        irr_4_m = irr_all_c.eq(4)
    
        irr_1_c = irr_all_c.updateMask(irr_1_m).rename(['irr_status_1'])
        irr_2_c = irr_all_c.updateMask(irr_2_m).rename(['irr_status_2'])
        irr_3_c = irr_all_c.updateMask(irr_3_m).rename(['irr_status_3'])
        irr_4_c = irr_all_c.updateMask(irr_4_m).rename(['irr_status_4'])

        # stack all bands
        all_bnds = irr_all_c.addBands(irr_1_c).addBands(irr_2_c).addBands(irr_3_c).addBands(irr_4_c)


        # images already masked to max irr extent raster so no need to add the updateMask here again
        stats_out = (
            all_bnds
                .reduceRegions(
                    collection=field_bound,
                    reducer=(
                        ee.Reducer.mode().unweighted()
                            .combine(ee.Reducer.count().unweighted(), 'irr_status_1_', False)
                            .combine(ee.Reducer.count().unweighted(), 'irr_status_2_', False)
                            .combine(ee.Reducer.count().unweighted(), 'irr_status_3_', False)
                            .combine(ee.Reducer.count().unweighted(), 'irr_status_4_', False)
                    ),
                    scale=30,
                    tileScale=16
                )
                .map(removGeom)
        )

        def orgFeatColl(ftr):
            return ftr.set({
                f'ETOF_IRR_STATUS_{str(year)[2:]}_MODE': ftr.get('mode'),
                f'1_NOT_IRRIGATED_{str(year)[2:]}_COUNT': ftr.get('irr_status_1_count'),
                f'2_IRRIGATED_{str(year)[2:]}_COUNT': ftr.get('irr_status_2_count'),
                f'3_SHORTED_{str(year)[2:]}_COUNT': ftr.get('irr_status_3_count'),
                f'4_OTHER_{str(year)[2:]}_COUNT': ftr.get('irr_status_4_count'),
            })


    # map the function to format 
    stats_out_form  = ee.FeatureCollection(stats_out.map(orgFeatColl))

    # list of properties to export
    if variable == 'etof_irr_status':
        selector_list = ['OPENET_ID', 
                          f'ETOF_IRR_STATUS_{str(year)[2:]}_MODE',
                          f'1_NOT_IRRIGATED_{str(year)[2:]}_COUNT', f'2_IRRIGATED_{str(year)[2:]}_COUNT',
                          f'3_SHORTED_{str(year)[2:]}_COUNT', f'4_OTHER_{str(year)[2:]}_COUNT']
    else:
        selector_list = ['OPENET_ID', 'ACRES_ALL', f'ACRES_{dataset_dict[variable][1]}']
    
    # Export tasks
    if export_location == 'google_drive':

        # Export a CSV file to Google Drive.
        out_table_task = ee.batch.Export.table.toDrive(**{
            'collection': stats_out_form,
            'description': f'OR_Field_Bound_Summary_Export_{variable}_{year}',
            'fileNamePrefix': f'or_field_summaries_{year}_{variable}',
            'fileFormat': 'CSV',
            'selectors': selector_list,
        })

    elif export_location == 'cloud_storage':
        
        # Export a CSV file to Cloud Storage.
        out_table_task = ee.batch.Export.table.toCloudStorage(**{
            'collection': stats_out_form,
            'description': f'OR_Field_Bound_Summary_Export_{variable}_{year}',
            'bucket': gcloud_path.split('/')[0],
            'fileNamePrefix': f'{gcloud_path}/or_field_summaries_{year}_{variable}',
            'fileFormat': 'CSV',
            'selectors': selector_list,
         })

    else:
        print('wrong export location setting, please check the parameter')
        continue

    # start the task
    out_table_task.start()

    # print the status of the task
    # print(out_table_task.status()["id"])
    
    print(f'task started for {variable} {year}')

## 4. **static** HUC8 and HUC12 attributes

In [ ]:
# function to set the field-centroid as a property
def addProp(ftr):
    return ftr.set({
        'centroid': ftr.geometry().centroid(),
    })

# function to add date properties to images
def addDates(img):
    img_date = ee.Date(img.get('system:time_start'))
    return img.set('date',img_date.format('yyyy-MM-dd'))

# simple join function for images
def joinFunc(img):
    return ee.Image.cat(img.get('primary'), img.get('secondary'))
    
# function to remove field-boundary geometries from zonal stats output
def removGeom(ftr):
    return ftr.setGeometry(None)

# field boundary assetID on GEE
if test_flag:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
            .filter(ee.Filter.eq('OPENET_ID', 'ORx_155121'))
    )
else:
    field_bound_pre = (
        ee.FeatureCollection('projects/openet/field_boundaries/Oregon_Hyd/Oregon_Hyd_Area_Ag_Boundaries_20240501')
            .select(['OPENET_ID'])
    )
    
# map the centroid function over the field boundary feature collection
field_bound = field_bound_pre.map(addProp)

# HUC feature collections
huc8 = (
    ee.FeatureCollection("USGS/WBD/2017/HUC08")
        .select(['name', 'huc8'],['huc8_name', 'huc8_code'])
)

huc12 = (
    ee.FeatureCollection("USGS/WBD/2017/HUC12")
        .select(['name', 'huc12'],['huc12_name', 'huc12_code'])
)

# Define a spatial filter as geometries that intersect, using the field centroid
spatialFilter = ee.Filter.intersects(
    leftField='centroid',
    rightField='.geo',
    maxError=10
)

# Define a save all joins for hucs
saveHucJoin8 = ee.Join.saveAll(
  matchesKey='HUC8r'
)

saveHucJoin12 = ee.Join.saveAll(
  matchesKey='HUC12r'
)

### Process one join at a time
# Apply the join.
field_bound_huc1 = saveHucJoin8.apply(field_bound, huc8, spatialFilter)

# adding id property of HUC feature to root feature
def hucProp8(ftr):
    hucid = ee.Feature(ee.List(ftr.get("HUC8r")).get(0)).get('huc8_code')
    hucna = ee.Feature(ee.List(ftr.get('HUC8r')).get(0)).get('huc8_name')
    return ftr.set({
        'HUC8': hucid,
        'HUC8_name': hucna,
        'HUC8r': None
    })

field_bound_out_huc1 = field_bound_huc1.map(hucProp8)


# Apply the join.
field_bound_huc2 = saveHucJoin12.apply(field_bound_out_huc1, huc12, spatialFilter)

# adding id property of HUC feature to root feature
def hucProp12(ftr):
    hucid = ee.Feature(ee.List(ftr.get("HUC12r")).get(0)).get('huc12_code')
    hucna = ee.Feature(ee.List(ftr.get('HUC12r')).get(0)).get('huc12_name')
    return ftr.set({
        'HUC12': hucid,
        'HUC12_name': hucna,
        'HUC12r': None
    })

field_bound_out = field_bound_huc2.map(hucProp12)


# list of properties to export
selector_list = ['OPENET_ID', 'HUC8_name', 'HUC8', 'HUC12_name','HUC12']

# Export tasks
if export_location == 'google_drive':

    # Export a CSV file to Google Drive.
    out_table_task = ee.batch.Export.table.toDrive(**{
        'collection': field_bound_out,
        'description': 'OR_Field_Bound_Summary_Export_HUC_attributes',
        'folder': gdrive_folder,
        'fileNamePrefix': 'or_field_summaries_huc_attributes',
        'fileFormat': 'CSV',
        'selectors': selector_list,
    })

elif export_location == 'cloud_storage':
    
    # Export a CSV file to Cloud Storage.
    out_table_task = ee.batch.Export.table.toCloudStorage(**{
        'collection': field_bound_out,
        'description': 'OR_Field_Bound_Summary_Export_HUC_attributes',
        'bucket': gcloud_path.split('/')[0],
        'fileNamePrefix': f'{gcloud_path}/or_field_summaries_huc_attributes',
        'fileFormat': 'CSV',
        'selectors': selector_list,
     })

else:
    print('wrong export location setting, please check the parameter')

# start the task
out_table_task.start()

# print the status of the task
# print(out_table_task.status()["id"])

print(f'task started for exporting field-level HUC8/HUC12 attributes')